<a href="https://colab.research.google.com/github/TevinAchong/info3604-project/blob/tinyBERT/tinyBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!/usr/bin/env python3
!pip install transformers
import tensorflow as tf
from transformers import *
import numpy as np
import pandas as pd
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 501kB 610kB/s 
     |████████████████████████████████| 3.7MB 52.0MB/s 
     |████████████████████████████████| 870kB 42.2MB/s 
     |████████████████████████████████| 1.0MB 26.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=f2ce76454463586480cfa611eb455fc3019a3b4f3fe63fb720a3b89825fa2b49
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [5]:
# Load dataset
df = pd.read_csv('IMDB_Dataset.csv', skiprows=1, header=None)
print(df.head())
df[1] = df[1].map({'positive': 1, 'negative': 0})
print(df.head())

df=df[:1500]

                                                   0         1
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
                                                   0  1
0  One of the other reviewers has mentioned that ...  1
1  A wonderful little production. <br /><br />The...  1
2  I thought this was a wonderful way to spend ti...  1
3  Basically there's a family where a little boy ...  0
4  Petter Mattei's "Love in the Time of Money" is...  1


In [0]:
# Load tokenizer and tinyBERT
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=True)
config = BertConfig.from_json_file('./6L_768D_FinalModel/CoLA/config.json')
model = BertModel.from_pretrained('./6L_768D_FinalModel/CoLA/pytorch_model.bin', config=config)

In [0]:
tokenized = df[0].apply((lambda x: tokenizer.encode(x[0:511], add_special_tokens=True)))

In [0]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

In [0]:
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

In [0]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [0]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

In [0]:
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [0]:
features = last_hidden_states[0][:,0,:].numpy()

labels = df[1]
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

In [0]:
print(lr_clf.score(test_features, test_labels))